# 1. Baseline: tf-idf + Naive Bayes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("train.csv")
dataset.head()

,id,name,tare
0,0,Котлеты МЛМ из говядины 335г,коробка
1,1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка
2,2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик
3,3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер
4,4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы


Для начала посмотрим на распределение категорий:

In [3]:
dataset["tare"].value_counts()

пакет без формы                   9028
бутылка                           7474
коробка                           4196
пакет прямоугольный               3501
обертка                           3217
банка неметаллическая             2238
стаканчик                         2070
банка металлическая               1837
вакуумная упаковка                1071
усадочная упаковка                 993
контейнер                          884
пачка                              691
лоток                              628
туба                               589
гофрокороб                         419
колбасная оболочка                 396
тортница                           324
без упаковки                       322
упаковка с газовым наполнением     289
ведро                              253
ячеистая упаковка                  228
Name: tare, dtype: int64

Распределение далеко от равномерного, поэтому важно использовать `stratify` для `train_test_split` и кросс-валидации

In [4]:
from sklearn.model_selection import train_test_split

train_part, test_part = train_test_split(dataset, test_size=0.2, random_state=0, stratify=dataset["tare"])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline_tfidf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                              ('nb', MultinomialNB())])

pipeline_tfidf_nb.fit(train_part["name"], train_part["tare"])

predicted = pipeline_tfidf_nb.predict(test_part["name"])
np.mean(predicted == test_part["tare"])

0.7124231242312423

Для начала неплохо, но даже из Naive Bayes можно выжать гораздо больше, подобрав правильные гиперпараметры.
Применим для этого Grid Search с кросс-валидацией:

In [6]:
from sklearn.model_selection import GridSearchCV

parameters_nb = {'tfidf__ngram_range': [(1, 1), (1, 2)],
                 'tfidf__use_idf': (True, False),
                 'nb__alpha': (1.0, 1e-1, 1e-2, 1e-3, 0),
                 }

gs_nb = GridSearchCV(pipeline_tfidf_nb, parameters_nb, scoring="accuracy", n_jobs=-1, cv=5, verbose=10,
                     return_train_score=True)
gs_nb = gs_nb.fit(dataset["name"], dataset["tare"])


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished
C:\Users\mrand\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [7]:
gs_nb.best_score_

0.8654054320015745

In [8]:
pd.DataFrame(gs_nb.cv_results_)[["mean_test_score",
                                 "param_tfidf__ngram_range",
                                 "param_tfidf__use_idf",
                                 "param_nb__alpha"]].sort_values(by=["mean_test_score"], ascending=False)

,mean_test_score,param_tfidf__ngram_range,param_tfidf__use_idf,param_nb__alpha
19,0.865405,"(1, 2)",False,0
10,0.864298,"(1, 2)",True,0.01
14,0.864077,"(1, 2)",True,0.001
18,0.863339,"(1, 2)",True,0
15,0.863265,"(1, 2)",False,0.001
11,0.859919,"(1, 2)",False,0.01
6,0.843486,"(1, 2)",True,0.1
8,0.828651,"(1, 1)",True,0.01
12,0.827273,"(1, 1)",True,0.001
13,0.826215,"(1, 1)",False,0.001


Уже гораздо лучше. Ключевыми оказались
* использование n-грамм
* правильный подбор `alpha` (additive smoothing parameter) для MultinomialNB

# 2. SVM

Время попробовать еще одну популярную модель для классификации текста - SVM, которая зачастую считается более "емкой"

In [9]:
from sklearn.linear_model import SGDClassifier

pipeline_tfidf_svm = Pipeline([('tfidf', TfidfVectorizer()),
                               ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=0.00001,
                                                     random_state=0, max_iter=15, class_weight=None)),
                               ])

parameters_svm = {'tfidf__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'svm__alpha': (1e-4, 1e-5, 1e-6),
                  'svm__class_weight': (None, "balanced")
                  }

gs_svm = GridSearchCV(pipeline_tfidf_svm, parameters_svm, scoring="accuracy", n_jobs=-1, cv=5, verbose=10,
                      return_train_score=True)
gs_svm = gs_svm.fit(dataset["name"], dataset["tare"])


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.1min finished


In [10]:
gs_svm.best_score_

0.8817407990553041

In [11]:
pd.DataFrame(gs_svm.cv_results_)[["mean_test_score",
                                  "param_tfidf__ngram_range",
                                  "param_tfidf__use_idf",
                                  "param_svm__alpha",
                                  "param_svm__class_weight"]].sort_values(by=["mean_test_score"], ascending=False)


,mean_test_score,param_tfidf__ngram_range,param_tfidf__use_idf,param_svm__alpha,param_svm__class_weight
10,0.881741,"(1, 2)",True,1e-05,None
11,0.878616,"(1, 2)",False,1e-05,None
18,0.876402,"(1, 2)",True,1e-06,None
14,0.874237,"(1, 2)",True,1e-05,balanced
19,0.872171,"(1, 2)",False,1e-06,None
22,0.871433,"(1, 2)",True,1e-06,balanced
23,0.870277,"(1, 2)",False,1e-06,balanced
15,0.866857,"(1, 2)",False,1e-05,balanced
8,0.865332,"(1, 1)",True,1e-05,None
9,0.859304,"(1, 1)",False,1e-05,None


Опять же важно использование n-грамм а также правильный выбор коэффициента регуляризации (`alpha`)

Похоже мы вплотную приблизились к ограничениям самих данных (и их представления).
На самом деле, названия товаров зачастую очень "грязные", и им не помешал бы препроцессинг

# 3. Очистка данных и препроцессинг

Для начала как работает встроенный tokenizer, который используется в `TfidfVectorizer`:
1. Пунктуация не воспринимается (запятые, точки, слэши... используются как разделители наравне с пробелами)
2. Используется `token_pattern=r"(?u)\b\w\w+\b"` то есть воспринимаются только токены от 2х символов

Это приводит например к тому, что обозначение **ж/б**, которое явно указывает (в подавляющем большинстве случаев) на тару *банка металлическая*, после стандартного tokenizer'а просто исчезает. Кроме того, возникают проблемы с вещественными числами (у которых разделители точки и запятые).


### Что бросается в глаза в самих названиях:
1. Присутствие как точек так и запятых в качестве разделителей для вещественных чисел (**12.5** vs **12,5**)
2. Различные форматы для веса, объема, процентов:
    * **0.5кг**, **0.5 кг**, **500г**, **500 г**, **500гр**, **500 гр**
    * **0.5л**, **0.5 л**, **500мл**, **500 мл**
    * **0.5%**, **0.5 %**
3. Обратный слэш "**\\**" вместо прямого: **ж\б** vs **ж/б**
4. **ЗефирДляДесертов125г** - camelCase вместо пробелов (стандартный tokenizer воспринимает это как один токен)
5. **1.8\*20** vs **1.8х20** (русская "**х**") vs **1.8x20** (латинская "**x**")

Напишем фунцию `cleanse_product_name`, которая принимает название товара и возвращает его очищенную версию:

In [12]:
import re

# переходы "aA", "Aa" (но не в начале слова), "a9", "A9", "9a", "9A"
camel_case_re = re.compile(r"((?<=[а-я])[А-Я]|(?<!\b)[А-Я](?=[а-я])|(?<=[А-Яа-я])[0-9]|(?<=[0-9])[А-Яа-я])")

# русская и латинская "x" между числами
times_re = re.compile(r"(?<=[0-9])[хХxX](?=[0-9])")

# string.punctuation без [.,%] но с [«»]
redundant_punctuation_re = re.compile(r'[!"#$&\'()*+\-/:;<=>?@\[\\\]^_`{|}~«»]')

# Для различных форматов веса, объема и процентов
weight_re = re.compile(r"(?P<number>[0-9]+([.][0-9]+)?)[ ]*(?P<units>(гр?|кг))")
volume_re = re.compile(r"(?P<number>[0-9]+([.][0-9]+)?)[ ]*(?P<units>(м?л))")
percent_re = re.compile(r"(?P<number>[0-9]+([.][0-9]+)?)[ ]*(?P<units>(%))")

dots_but_not_inside_numbers_re = re.compile(r"(?<![0-9])\.|\.(?![0-9])")

redundant_slashes_re = re.compile(r"(?<= )/|(?<=\A)/|/(?= )|/(?=\Z)")


# "0.5кг", "0.5 кг", "500г", "500 г", "500гр", "500 гр" -> " 500 гр "
def format_weight_value(match_object):
    number = float(match_object.group("number"))
    units = match_object.group("units")

    if units == "кг":
        number *= 1000

    number = int(round(number))
    return " " + str(number) + " гр "


# "0.5л", "0.5 л", "500мл", "500 мл" -> " 0.5 л "
def format_volume_value(match_object):
    number = float(match_object.group("number"))
    units = match_object.group("units")

    if units == "мл":
        number /= 1000
    return " " + str(number).rstrip('0').rstrip('.') + " л "


# "0.5%", "0.5 %" -> " 0.5 % "
def format_percent_value(match_object):
    number = float(match_object.group("number"))
    return " " + str(number).rstrip('0').rstrip('.') + " % "



def cleanse_product_name(name: str):
    name = name.replace("ё", "е")
    name = name.replace("Ё", "Е")

    # "1.8x20" -> "1.8*20"
    name = re.sub(times_re, "*", name)

    # Заменить camelCase на разделение пробелами
    # Example: "ЗефирДляДесертов125г" -> "Зефир Для Десертов 125 г"
    name = re.sub(camel_case_re, r' \1', name)

    # Теперь у заглавных букв нет пользы
    name = name.lower()

    # Убрать пунктуацию кроме . , %
    name = re.sub(redundant_punctuation_re, " ", name)

    # Чтобы не возиться с запятыми в вещественных числах
    name = name.replace(",", ".")

    name = re.sub(weight_re, format_weight_value, name)
    name = re.sub(volume_re, format_volume_value, name)
    name = re.sub(percent_re, format_percent_value, name)

    # удалить "." если не внутри вещественного числа
    name = re.sub(dots_but_not_inside_numbers_re, " ", name)

    return name


Сделаем препроцессинг функцией `cleanse_product_name` для всего dataset'а:

In [13]:
dataset_names_cleansed = dataset["name"].apply(cleanse_product_name)

Снова протестируем SVM (с лучшими найденными гиперпараметрами), но уже на очищенных данных. Необходимо изменить `token_pattern` на `r"(?u)\b\w+\b"`, чтобы он учитывал однобуквенные токены.

"**ж/б**" (и "**ж\б**") превратятся в "**ж б**" и захватятся соответствующей 2-граммой

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

pipeline_tfidf_svm = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2),
                                                         token_pattern=r"(?u)\b\w+\b",
                                                         use_idf=True)),
                               ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=0.00001,
                                                     random_state=0, max_iter=15, class_weight=None)),
                               ])

cross_val_score(pipeline_tfidf_svm, dataset_names_cleansed, dataset["tare"], scoring="accuracy", cv=5, n_jobs=-1).mean()


0.8963548708888075

\+ 0.015 к accuracy после очистки данных

И еще одно возможное улучшение, которое напрашивается в фазу препроцессинга - это

### Стемминг

In [15]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()

def stem_all_words(name: str):
    return " ".join([stemmer.stem(token) for token in name.split()])

dataset_names_stemmed = dataset_names_cleansed.apply(stem_all_words)


In [16]:
cross_val_score(pipeline_tfidf_svm, dataset_names_stemmed, dataset["tare"], scoring="accuracy", cv=5, n_jobs=-1).mean()

0.8975361380109307

Улучшение совсем небольшое, но все же есть (+ 0.001 к accuracy)

# 4. Confusion Matrix

Итак, лучшая модель в итоге следующая:
1. Очистить названия товаров с помощью `cleanse_product_name`
2. Произвести стемминг
3. 
```python
TfidfVectorizer(ngram_range=(1, 2), token_pattern=r"(?u)\b\w+\b", use_idf=True)
SGDClassifier(loss='hinge', penalty='l2', alpha=0.00001, random_state=0, max_iter=15, class_weight=None)
```

Это можно записать одним Pipeline'ом:

In [17]:
pipeline_tfidf_svm_prep = Pipeline(
    [('tfidf', TfidfVectorizer(preprocessor=lambda name: stem_all_words(cleanse_product_name(name)),
                               ngram_range=(1, 2), token_pattern=r"(?u)\b\w+\b", use_idf=True)),
     ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=0.00001,
                           random_state=0, max_iter=15, class_weight=None)),
     ])


Построим confusion_matrix для нашего первоначального train_test разбиения:


In [18]:
test_classes_counts = test_part["tare"].value_counts()
test_classes_names = np.array(test_classes_counts.index)
total_classes = len(test_classes_counts)

In [19]:
pipeline_tfidf_svm_prep.fit(train_part["name"], train_part["tare"])

predicted = pipeline_tfidf_svm_prep.predict(test_part["name"])
np.mean(predicted == test_part["tare"])


0.8952029520295203

In [20]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=test_part["tare"], y_pred=predicted, labels=test_classes_names)
cm

array([[1638,    6,   27,   38,   39,    2,    8,    1,    6,   17,    2,
           2,    3,    3,    9,    1,    1,    2,    0,    1,    0],
       [   2, 1471,    2,   10,    0,    3,    2,    5,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [  41,    7,  730,    8,    7,    8,    4,    1,    1,    4,   12,
           3,    2,    1,    3,    1,    2,    1,    0,    0,    3],
       [  54,    8,    4,  607,    9,    5,    1,    0,    2,    4,    1,
           0,    2,    1,    0,    0,    0,    1,    0,    1,    0],
       [  64,    4,    8,    9,  535,    4,    1,    0,    4,    4,    1,
           1,    2,    0,    0,    4,    0,    1,    1,    0,    0],
       [   2,    1,    3,    2,    1,  426,    3,    3,    2,    0,    1,
           0,    0,    2,    0,    1,    0,    0,    1,    0,    0],
       [   5,    4,    0,    3,    6,    0,  393,    1,    0,    0,    0,
           0,    1,    1,    0,    0,    0,    0,    0,    0,    0],

Для каждого типа упаковки выведем наиболее популярные предсказания (>=5% случаев)

In [21]:
for true_class_id in range(total_classes):
    true_class_name = test_classes_names[true_class_id]
    true_class_count = test_classes_counts[true_class_name]
    
    print('Для настоящей упаковки "{0}" ({1} тестовых экземпляров) были предложены:'.format(true_class_name, true_class_count))
    for pred_class_id in range(total_classes):
        percent = cm[true_class_id, pred_class_id] / true_class_count * 100
        if percent >= 5:
            pred_class_name = test_classes_names[pred_class_id]
            print('\t"{0}" в {1:.2f} % случаев ({2} раз)'.format(pred_class_name, percent, cm[true_class_id, pred_class_id]))


Для настоящей упаковки "пакет без формы" (1806 тестовых экземпляров) были предложены:
	"пакет без формы" в 90.70 % случаев (1638 раз)
Для настоящей упаковки "бутылка" (1495 тестовых экземпляров) были предложены:
	"бутылка" в 98.39 % случаев (1471 раз)
Для настоящей упаковки "коробка" (839 тестовых экземпляров) были предложены:
	"коробка" в 87.01 % случаев (730 раз)
Для настоящей упаковки "пакет прямоугольный" (700 тестовых экземпляров) были предложены:
	"пакет без формы" в 7.71 % случаев (54 раз)
	"пакет прямоугольный" в 86.71 % случаев (607 раз)
Для настоящей упаковки "обертка" (643 тестовых экземпляров) были предложены:
	"пакет без формы" в 9.95 % случаев (64 раз)
	"обертка" в 83.20 % случаев (535 раз)
Для настоящей упаковки "банка неметаллическая" (448 тестовых экземпляров) были предложены:
	"банка неметаллическая" в 95.09 % случаев (426 раз)
Для настоящей упаковки "стаканчик" (414 тестовых экземпляров) были предложены:
	"стаканчик" в 94.93 % случаев (393 раз)
Для настоящей упаковки

## 5. Предсказание для test.csv

In [22]:
test_csv_data = pd.read_csv("test.csv")

In [23]:
pipeline_tfidf_svm_prep = Pipeline(
    [('tfidf', TfidfVectorizer(preprocessor=lambda name: stem_all_words(cleanse_product_name(name)),
                               ngram_range=(1, 2), token_pattern=r"(?u)\b\w+\b", use_idf=True)),
     ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=0.00001,
                           random_state=0, max_iter=15, class_weight=None)),
     ])

pipeline_tfidf_svm_prep.fit(dataset["name"], dataset["tare"])

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function <lambd...lty='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [24]:
predicted_for_test_csv = pipeline_tfidf_svm_prep.predict(test_csv_data["name"])

result = pd.DataFrame({"id": test_csv_data["id"].values, "tare": predicted_for_test_csv})
result.head()

,id,name
0,40648,бутылка
1,40649,гофрокороб
2,40650,коробка
3,40651,пакет прямоугольный
4,40652,пачка


In [25]:
result.to_csv("prediction.csv", index=False)